<a href="https://colab.research.google.com/github/DavidBarral11/Mineria_de_datos_USAC/blob/main/RED_NEURONAL_FALTA_JUDICIAL_SEXO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split



Saving base-faltas-judiciales-2024.xlsx to base-faltas-judiciales-2024.xlsx


In [2]:

filename = list(uploaded.keys())[0]
print("Archivo cargado:", filename)

data = pd.read_excel(filename)

data.head()


Archivo cargado: base-faltas-judiciales-2024.xlsx


,num_corre,depto_boleta,muni_boleta,mes_boleta,año_boleta,falta_inf,sexo_inf,edad_inf,grupo_etnico_inf,est_conyugal_inf,...,g_primarios,area_geo_inf,est_ebriedad_inf,g_edad_80ymas,g_edad_60ymas,edad_quinquenales,depto_nacimiento_inf,nacionalidad_inf,subg_principales,gran_grupos
0,1,1,102,4,2024,4,1,22,2,1,...,9621,2,2,3,3,5,1,1,96,9
1,2,1,102,4,2024,3,1,20,2,1,...,8322,1,1,3,3,5,1,1,83,8
2,3,1,102,4,2024,3,1,18,2,1,...,9998,1,1,2,2,4,1,1,98,11
3,4,1,102,4,2024,4,1,18,2,1,...,9998,1,1,2,2,4,1,1,98,11
4,5,1,102,4,2024,3,1,21,2,1,...,7122,1,1,3,3,5,1,1,71,7


In [3]:
data['edad_inf'] = data['edad_inf'].astype(str)
mask_9s = data['edad_inf'].str.fullmatch(r'9+')
data.loc[mask_9s, 'edad_inf_limpia'] = np.nan
data.loc[~mask_9s, 'edad_inf_limpia'] = pd.to_numeric(data.loc[~mask_9s, 'edad_inf'], errors='coerce')

data['sexo_inf'] = pd.to_numeric(data['sexo_inf'], errors='coerce')
data['est_ebriedad_inf'] = pd.to_numeric(data['est_ebriedad_inf'], errors='coerce')

data_prop = data[data['falta_inf'] == 3].copy()

data_prop = data_prop.dropna(subset=['sexo_inf'])

data_prop['es_hombre'] = data_prop['sexo_inf'].apply(lambda x: 1 if x == 1 else 0)

data_prop = data_prop.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
X = data_prop[[
    'edad_inf_limpia',
    'grupo_etnico_inf',
    'est_conyugal_inf',
    'cond_alfabetismo_inf',
    'niv_escolaridad_inf',
    'area_geo_inf',
    'est_ebriedad_inf',
    'gran_grupos'
]]

X = X.fillna(0)

y = data_prop['es_hombre']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 289 (1.13 KB)

 Trainable params: 289 (1.13 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8331 - loss: 0.4563 - val_accuracy: 0.9184 - val_loss: 0.3422
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9229 - loss: 0.3082 - val_accuracy: 0.9184 - val_loss: 0.3203
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9231 - loss: 0.2940 - val_accuracy: 0.9184 - val_loss: 0.3097
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9130 - loss: 0.3131 - val_accuracy: 0.9184 - val_loss: 0.3042
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9179 - loss: 0.2934 - val_accuracy: 0.9184 - val_loss: 0.3009
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9204 - loss: 0.2883 - val_accuracy: 0.9184 - val_loss: 0.3025
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9207 - loss: 0.2793 - val_accuracy: 0.9184 - val_loss: 0.2989
Epoch 8/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9136 - loss: 0.2975 - val_accuracy: 0.9184 - v

In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9308 - loss: 0.1974
Loss: 0.2137, Accuracy: 0.9272


In [8]:
hecho = np.array([[35, 1, 3, 1, 2, 1, 2, 9]], dtype='float32')

res = model.predict(hecho)
print("Probabilidad estimada de que el infractor sea hombre:", res)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Probabilidad estimada de que el infractor sea hombre: [[0.97735894]]
